In [2]:
from google.colab import drive
drive.mount('/content/drive')
!pip install deepchem rdkit matplotlib networkx

import deepchem as dc
from rdkit import Chem
from rdkit.Chem import Draw
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from deepchem.feat import MolGraphConvFeaturizer
from deepchem.feat import CircularFingerprint
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from deepchem.feat import ConvMolFeaturizer
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
import deepchem as dc
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate


#from methods_fingerprints import *
from drive.MyDrive.code.methods import *
from drive.MyDrive.code.methods_fingerprints import *

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 8.3 MB/s eta 0:00:00


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


In [3]:
smiles_tr,smiles_test, logSweeter_tr, logSweeter_test, families_tr, families_test = load_data("/content/drive/MyDrive/code/SweetnersDB.xlsx")
X_tr,X_test,scaler_X = prepare_fingerprint(smiles_tr,smiles_test)
y_tr,y_test,scaler_y = prepare_targets(logSweeter_tr,logSweeter_test)
onehot_tr,onehot_test = prepare_famille_chimique(families_tr, families_test)

Index(['Name', 'logSw', 'Chemical family', 'Dataset', 'Smiles'], dtype='object')


[13:58:56] DEPRECATION WARNING: please use MorganGenerator
[13:58:56] DEPRECATION WARNING: please use MorganGenerator
[13:58:56] DEPRECATION WARNING: please use MorganGenerator
[13:58:56] DEPRECATION WARNING: please use MorganGenerator
[13:58:56] DEPRECATION WARNING: please use MorganGenerator
[13:58:56] DEPRECATION WARNING: please use MorganGenerator
[13:58:56] DEPRECATION WARNING: please use MorganGenerator
[13:58:56] DEPRECATION WARNING: please use MorganGenerator
[13:58:56] DEPRECATION WARNING: please use MorganGenerator
[13:58:56] DEPRECATION WARNING: please use MorganGenerator
[13:58:56] DEPRECATION WARNING: please use MorganGenerator
[13:58:56] DEPRECATION WARNING: please use MorganGenerator
[13:58:56] DEPRECATION WARNING: please use MorganGenerator
[13:58:56] DEPRECATION WARNING: please use MorganGenerator
[13:58:56] DEPRECATION WARNING: please use MorganGenerator
[13:58:56] DEPRECATION WARNING: please use MorganGenerator
[13:58:56] DEPRECATION WARNING: please use MorganGenerat

In [4]:
input_fingerprint = Input(shape=(1024,), name='fingerprint_input')
input_famille = Input(shape=(10,), name='famille_input')

# Combiner les entrées
concat = Concatenate()([input_fingerprint, input_famille])

# Ajouter des couches denses
dense = Dense(512, activation='relu')(concat)
output = Dense(256, activation='relu')(dense)

# Créer le modèle
model = Model(inputs=[input_fingerprint, input_famille], outputs=output)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ fingerprint_input         │ (None, 1024)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ famille_input             │ (None, 10)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 1034)           │              0 │ fingerprint_input[0][… │
│                           │                        │                │ famille_input[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 512)            │        529,920 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 256)            │        131,328 │ dense[0][0]            │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 661,248 (2.52 MB)

 Trainable params: 661,248 (2.52 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# Faire une prédiction avec le modèle
predictions_tr = model.predict([X_tr, onehot_tr])
print("\nPrédictions :")
print(predictions_tr)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step

Prédictions :
[[0.16786122 0.         0.21113802 ... 0.30726385 0.         0.        ]
 [0.         0.09198359 0.6877161  ... 0.         0.         0.        ]
 [0.         0.01893003 0.15676087 ... 0.0384301  0.         0.        ]
 ...
 [0.         0.07443705 1.2439222  ... 0.27978677 0.         0.        ]
 [0.         0.         2.7773216  ... 0.         1.2316117  0.9183395 ]
 [0.         0.         1.6012222  ... 1.0465606  0.         0.        ]]


In [6]:
# Faire une prédiction avec le modèle
predictions_test = model.predict([X_test, onehot_test])
print("\nPrédictions :")
print(predictions_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step

Prédictions :
[[0.         0.         0.09472081 ... 0.0743466  0.         0.        ]
 [0.         0.         0.45379233 ... 0.25348747 0.6155244  0.31187683]
 [0.3479626  0.17043582 1.0200567  ... 1.3626283  0.         0.01010448]
 ...
 [0.         0.         1.2808363  ... 0.37403578 1.5384812  1.2411124 ]
 [0.         0.         1.2071637  ... 0.09874377 0.63008004 0.252342  ]
 [0.22198272 0.         2.0848043  ... 0.16431224 1.1966542  0.13457692]]


# scoring reg log

In [7]:
accuracy,conf_matrix,class_report,y_pred,target_test = scoring_GridSearch(y_tr,y_test,predictions_tr,predictions_test,0,"LogReg")

Fitting 5 folds for each of 180 candidates, totalling 900 fits
[CV] END .......C=0.01, l1_ratio=0, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .......C=0.01, l1_ratio=0, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .......C=0.01, l1_ratio=0, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .......C=0.01, l1_ratio=0, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .......C=0.01, l1_ratio=0, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...C=0.01, l1_ratio=0, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END ...C=0.01, l1_ratio=0, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END ...C=0.01, l1_ratio=0, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END ...C=0.01, l1_ratio=0, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END ...C=0.01, l1_ratio=0, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END ........C=0.01, l1_ratio=0, penalty=l1, solver=saga; total time=   0.0s
[CV] END ........C=0.01, l1_ratio=0, penalty=l

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_

[CV] END .......C=0.01, l1_ratio=0, penalty=l2, solver=lbfgs; total time=   0.0s
[CV] END .......C=0.01, l1_ratio=0, penalty=l2, solver=lbfgs; total time=   0.0s
[CV] END .......C=0.01, l1_ratio=0, penalty=l2, solver=lbfgs; total time=   0.0s
[CV] END .......C=0.01, l1_ratio=0, penalty=l2, solver=lbfgs; total time=   0.0s
[CV] END ...C=0.01, l1_ratio=0, penalty=l2, solver=liblinear; total time=   0.0s
[CV] END ...C=0.01, l1_ratio=0, penalty=l2, solver=liblinear; total time=   0.0s
[CV] END ...C=0.01, l1_ratio=0, penalty=l2, solver=liblinear; total time=   0.0s
[CV] END ...C=0.01, l1_ratio=0, penalty=l2, solver=liblinear; total time=   0.0s
[CV] END ...C=0.01, l1_ratio=0, penalty=l2, solver=liblinear; total time=   0.0s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_

[CV] END ........C=0.01, l1_ratio=0, penalty=l2, solver=saga; total time=   0.4s


...

[CV] END .......C=100, l1_ratio=1, penalty=none, solver=saga; total time=   2.1s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=none)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1183: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


[CV] END .......C=100, l1_ratio=1, penalty=none, solver=saga; total time=   4.2s


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
300 fits failed out of a total of 900.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
75 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1169, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/

Best Parameters: {'C': 1, 'l1_ratio': 0.5, 'penalty': 'elasticnet', 'solver': 'saga'}
Best Cross-validation Score: 0.7417254901960785
Accuracy: 0.828125
Confusion Matrix:
[[16  5]
 [ 6 37]]
Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.76      0.74        21
           1       0.88      0.86      0.87        43

    accuracy                           0.83        64
   macro avg       0.80      0.81      0.81        64
weighted avg       0.83      0.83      0.83        64



# scoring decision tree

In [9]:
accuracy_tree,conf_matrix_tree,class_report_tree,y_pred_tree,target_test_tree = scoring_GridSearch(y_tr,y_test,predictions_tr,predictions_test,0,"DecisionTree")

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
[CV] END criterion=gini, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=gini, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=gini, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=gini, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=gini, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=gini, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=5; total time=   0.0s
[CV] END criterion=gini, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=5; total time=   0.0s
[CV] END criterion=gini, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_spli

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
360 fits failed out of a total of 1080.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
360 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 638, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/uti

# scoring random forest

In [10]:
accuracy_RF,conf_matrix_RF,class_report_RF,y_pred_RF,target_test_RF = scoring_GridSearch(y_tr,y_test,predictions_tr,predictions_test,0,"RandomF")

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
540 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
540 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 638, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/uti

Best Parameters: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
Best Cross-validation Score: 0.7182745098039216
Accuracy: 0.828125
Confusion Matrix:
[[12  9]
 [ 2 41]]
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.57      0.69        21
           1       0.82      0.95      0.88        43

    accuracy                           0.83        64
   macro avg       0.84      0.76      0.78        64
weighted avg       0.83      0.83      0.82        64

